In [6]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'blake-griffin', 'hasheem-thabeet', 'james-harden', 'tyreke-evans', 'jonny-flynn', 'stephen-curry',  
    'jordan-hill', 'demar-derozan', 'terrence-williams', 'gerald-henderson', 'tyler-hansbrough', 
    'earl-clark', 'austin-daye', 'james-johnson-2', 'jure-holiday', 'ty-lawson', 'jeff-teague', 
    'eric-maynor', 'darren-collison', 'demarre-carroll', 'wayne-ellington', 'toney-douglas', 'jeff-ayres', 
    'jermaine-taylor', 'dante-cunningham', 'dajuan-summers', 'sam-young',  'dejuan-blair', 'jon-brockman', 
    'derrick-brown', 'jodie-meeks', 'patrick-beverly', 'chase-budinger', 'nick-calathes', 
    'danny-green', 'taylor-griffin', 'goran-suton', 'jack-mcclinton', 'aj-price', 'robert-vaden', 
    'patrick-mills',  'robert-dozier']
   
player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2009')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2009'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)
  
table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

per40 = 40/table['MP']

table["Player Grade"] = ((table['PTS']*per40) + (table['TRB']*1.5*per40) + (table['AST']*2*per40) +
(table['BLK']*3*per40) + (table['STL']*3*per40) + (table['3P']*5*per40)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.25
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['Age']
table

,Name,Season,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,,Player Grade
1,Stephen Curry,2008-09,Davidson,-3.33,34,33.7,.454,3.8,9.9,0.387,0.876,4.4,5.6,2.5,0.2,3.7,2.4,28.6,,101.0
2,Blake Griffin,2008-09,Oklahoma,6.88,35,33.3,.654,0.1,0.2,0.375,0.590,14.4,2.3,1.1,1.2,3.3,2.5,22.7,,97.8
3,Tyreke Evans,2008-09,Memphis,4.63,37,29.0,.455,0.9,3.4,0.274,0.711,5.4,3.9,2.1,0.8,3.6,2.4,17.1,,95.1
4,Dejuan Blair,2008-09,Pitt,8.61,35,27.3,.593,0.0,0.0,0.000,0.605,12.3,1.2,1.5,1.0,1.3,2.7,15.7,,92.9
5,James Harden,2008-09,Arizona State,7.82,35,35.8,.489,1.7,4.7,0.356,0.756,5.6,4.2,1.7,0.3,3.4,2.8,20.1,,91.7
6,Nick Calathes,2008-09,Florida,3.88,36,33.3,.482,2.0,5.2,0.390,0.707,5.3,6.4,1.9,0.2,3.3,1.9,17.2,,91.5
7,Ty Lawson,2008-09,UNC,8.35,35,29.9,.532,1.5,3.1,0.472,0.798,3.0,6.6,2.1,0.1,1.9,1.7,16.6,,85.5
8,Jack Mcclinton,2008-09,Miami (FL),9.04,32,32.2,.449,3.2,7.0,0.453,0.885,3.1,2.8,0.8,0.0,2.6,1.6,19.3,,84.3
9,Danny Green,2008-09,UNC,8.35,38,27.4,.471,2.0,4.8,0.418,0.852,4.7,2.7,1.8,1.3,1.7,2.2,13.1,,82.9
10,Jeff Teague,2008-09,Wake Forest,7.36,31,32.0,.485,1.5,3.3,0.441,0.817,3.3,3.5,1.9,0.6,3.4,2.2,18.8,,82.1
